<a href="https://colab.research.google.com/github/E-pep/ThesisLitter1920/blob/master/LearnMobilenetV2Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/tutorials/images/transfer_learning


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
  !pip install tensorflow-gpu==2.1.0
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2

In [0]:
# standard it is TF 2.0.0 but better be sure
print(tf.__version__)

# **Data preprocessing**

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import pathlib
# Insert Path to the dataset
data_dir = pathlib.Path('Path to dataset')

# **Batch info klaarzetten**

In [0]:
image_count = len(list(data_dir.glob('*/*')))
image_count

In [0]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

In [0]:
BATCH_SIZE = image_count
IMG_HEIGHT = 160
IMG_WIDTH = 160
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [0]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

# **Load using tf.data**

In [0]:
# insert path to dataset
data_dir = pathlib.Path('path to dataset')
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
len(list(list_ds))

In [0]:
for f in list_ds.take(50):
  print(f.numpy())

In [0]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [0]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  try:
    img = tf.image.decode_jpeg(img, channels=3)
  except:
    print("fout bij:")
    print(img)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [0]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [0]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path)

In [0]:
for image, label in labeled_ds.take(5):
  try:
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
  except:
    print("kapot")

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [0]:
train_ds = prepare_for_training(labeled_ds)
#len(list(train_ds))
#image_batch, label_batch = next(iter(train_ds))
#len(list(train_ds))

In [0]:
image_batch, label_batch = next(iter(train_ds))

In [0]:
label_batch.numpy()

In [0]:
show_batch(image_batch.numpy(), label_batch.numpy())

# **Split data**

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((image_batch, label_batch))
#dataset = dataset.batch(4)

In [0]:
len(list(dataset))

In [0]:
dataset

In [0]:
# import tensorflow_datasets as tfds
# (raw_train, raw_validation, raw_test), metadata = dataset(split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
#     with_info=True,
#     as_supervised=True,
# )

train_size = int(0.8 * len(list(dataset)))
val_size = int(0.1 * len(list(dataset)))
test_size = int(0.1 * len(list(dataset)))

dataset = dataset.shuffle(1000)
raw_train = dataset.take(train_size)
raw_test = dataset.skip(train_size)
raw_validation = raw_test.skip(val_size)
raw_test = raw_test.take(test_size)

print(raw_train.take(1))
# train_batches = train_ds.shuffle(1000).batch(32)
# validation_batches = train_ds.batch(32)
# test_batches = train_ds.batch(32)
# train_batches.take(1)

In [0]:
train_size

In [0]:
len(list(raw_train))

In [0]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [0]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [0]:
len(list(train))

In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [0]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [0]:
len(list(train_batches))

In [0]:
for image_batch, label_batch in train_batches.take(1):
   pass

#image_batch.shape
image_batch.shape

# **Create the base model from the pre-trained convnets**

In [0]:
IMG_SIZE = 160

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

# **Testen om mobilenet als layers**

In [0]:
from tensorflow.keras.models import Model
def  Mymodel(backbone_model, classes):
    backbone = backbone_model
    backbone.trainable = False
    x = backbone.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(classes)(x)
    x = tf.keras.layers.Activation('softmax')(x)
    model = Model(inputs=backbone.input, outputs=x)
    return model

model = Mymodel(backbone_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet'),classes=2)
model.summary()

# **Compile the model**

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
len(model.trainable_variables)

# **Train the model**

In [0]:
initial_epochs = 20
validation_steps=6

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

In [0]:
len(list(validation_batches))

In [0]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [0]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# **Fine tuning**

In [0]:
base_model.trainable = True

In [0]:

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False


In [0]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])


In [0]:
model.summary()

In [0]:
len(model.trainable_variables)


In [0]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)


In [0]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [0]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [0]:
image_batch, label_batch = next(iter(train_ds))

In [0]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = CLASS_NAMES[predicted_id]

In [0]:
predicted_id

In [0]:
predicted_label_batch

# **View result**

In [0]:
def show_batch_testresult(image_batch, predicted_id, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      color = "green" if predicted_id[n] != label_batch[n][0] else "red"
      plt.title(CLASS_NAMES[predicted_id[n]], color=color)
      plt.axis('off')

In [0]:
label_batch[label_batch[n]==1][0]

In [0]:
CLASS_NAMES[0] = 'litter'
CLASS_NAMES[1] = 'non-litter'
show_batch_testresult(image_batch.numpy(), predicted_id, label_batch.numpy())

In [0]:
show_batch(image_batch.numpy(), label_batch.numpy())

# **Safe het model**

In [0]:
import time
t = time.time()

#insert path to save
export_path = "path to save".format(int(t))
tf.keras.models.save_model(model, export_path, save_format="h5")

export_path

Now confirm that we can reload it, and it still gives the same results:

In [0]:
reloaded = tf.keras.models.load_model("path where saved")

In [0]:
reloaded.summary()

In [0]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [0]:
abs(reloaded_result_batch - result_batch).max()